In [13]:
import pandas as pd
import numpy as np

import seaborn as sns
import math
from matplotlib import rcParams
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import datetime
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as po

# 서울 각 자치구의 재활용품 수거 현황 
* 재활용품 처리량이 많은 자치구에 대한 분석을 진행하고자 함
* 추후 직장인구, 생활인구 등을 같이 고려하면 좋을 듯

In [2]:
path = "../../데이터분석 공모전/2021 공공빅데이터 공모전/cup_sharing/"

In [495]:
trash = pd.read_csv(path+"서울시 쓰레기수거 현황 통계.csv")
trash

,기간,자치구,행정구역(A),행정구역(A).1,청소구역(B),청소구역(B).1,수거지인구율(B/A)(%),배출량(C),처리량(D),수거율(D/C)(%),처리방법,처리방법.1,처리방법.2,처리방법.3,처리방법.4
0,기간,자치구,면적,인구,면적,인구,수거지인구율(B/A)(%),배출량(C),처리량(D),수거율(D/C)(%),계,매립,소각,재활용,기타
1,2019,합계,605.24,10010983,605.24,10010983,100,47995.8,47995.8,100,47995.8,2037.1,2700.8,42235.6,1022.3
2,2019,종로구,23.91,161869,23.91,161869,100,882,882,100,882,52.4,93.7,735.3,0.6
3,2019,중구,9.96,136488,9.96,136488,100,1468.2,1468.2,100,1468.2,60.2,139.9,1268.0,0.1
4,2019,용산구,21.87,245185,21.87,245185,100,1140.8,1140.8,100,1140.8,48.5,73.6,1009.6,9.1
5,2019,성동구,16.86,308979,16.86,308979,100,2181.2,2181.2,100,2181.2,319.3,120.1,1442.1,299.7
6,2019,광진구,17.06,366972,17.06,366972,100,1379.4,1379.4,100,1379.4,25,63.5,1290.4,0.5
7,2019,동대문구,14.22,363023,14.22,363023,100,1367.2,1367.2,100,1367.2,23.2,109,1234.7,0.3
8,2019,중랑구,18.5,402024,18.5,402024,100,1585.9,1585.9,100,1585.9,27.4,96.7,1461.3,0.5
9,2019,성북구,24.57,454744,24.57,454744,100,1354.6,1354.6,100,1354.6,12.2,94,1247.6,0.8


In [499]:
recycle = trash[['기간','자치구','행정구역(A)','처리방법.3']].loc[2:]
recycle.columns = ['기간','자치구','면적','재활용']
recycle['면적'] = pd.to_numeric(recycle['면적'])
recycle['재활용'] = pd.to_numeric(recycle['재활용'])

# 재활용 쓰레기가 많은 순 (처리된 재활용의 수가 많은 순)

In [553]:
recycle['재활용/면적'] = recycle['재활용']/recycle['면적']
recycle = recycle.sort_values('재활용',ascending=False)
recycle

,기간,자치구,면적,재활용,재활용/면적
26,2019,강동구,24.59,4356.1,177.149248
23,2019,서초구,46.98,3566.8,75.921669
24,2019,강남구,39.50,3059.5,77.455696
20,2019,영등포구,24.55,2377.5,96.843177
17,2019,강서구,41.44,2213.8,53.421815
13,2019,은평구,29.71,2146.4,72.245035
25,2019,송파구,33.87,2041.2,60.265722
19,2019,금천구,13.02,1798.5,138.133641
15,2019,마포구,23.85,1753.4,73.517820
22,2019,관악구,29.57,1533.2,51.849848


# 면적 대비, 재활용 쓰레기가 많은 순 (처리된 재활용의 수가 많은 순)

In [501]:
recycle = recycle.sort_values('재활용/면적',ascending=False)
recycle

,기간,자치구,면적,재활용,재활용/면적
26,2019,강동구,24.59,4356.1,177.149248
19,2019,금천구,13.02,1798.5,138.133641
3,2019,중구,9.96,1268.0,127.309237
20,2019,영등포구,24.55,2377.5,96.843177
7,2019,동대문구,14.22,1234.7,86.828411
21,2019,동작구,16.35,1410.9,86.293578
5,2019,성동구,16.86,1442.1,85.533808
14,2019,서대문구,17.63,1490.7,84.554736
8,2019,중랑구,18.50,1461.3,78.989189
24,2019,강남구,39.50,3059.5,77.455696
